In [3]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="7" color="red">ch15. 데이터베이스연동</font></b>
# 1절 SQLite 데이터 베이스 연결
- SQLite 데이터 베이스는 별도의 DBMS 없어 SQL문 이용해서 DB액세스할 수 있도록 간단한 디스크 기반 DB
- C라이브러리
- SQLite는 프로토타입 생성기 사용
- 프로젝트 : 분석 -> 설계 -> 구현 -> 테스트 -> 고객에게 배포 -> 유지보수
                프로토타입(SQLite)   완제품(Oracle, MySQL, Maria,...)
- DB browser for SQLite 


In [6]:
import sqlite3
sqlite3.sqlite_version # 버전

'3.40.1'

In [7]:
import pandas as pd
pd.__version__

'1.5.3'

## 1.2 데이터 베이스 연결

- (1) 데이터베이스 연결
- (2) SQL 전송 객체 생성(cursor)
- (3) SQL 전송(cursor.execute()이용) 
- (4) cursor 해제 & 데이터베이스연결객체 해제


In [8]:
# (1)DB 연결 : sqlite로 연결시 DB파일이 없으면 빈 DB파일 생성, 있으면 연결
conn = sqlite3.connect('data/ch15_example.db')
conn

In [10]:
# (2) SQL 전송 객체 생성(cursor 객체)
cursor = conn.cursor()
cursor

In [14]:
cursor.execute('''
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
        )
''')

In [13]:
cursor.execute('DROP TABLE MEMBER')


In [17]:
cursor.execute('INSERT INTO MEMBER VALUES(\'홍길동\',20,\'h@h.com\')')
print('수행결과 행수 :',cursor.rowcount)

수행결과 행수 : 1


In [19]:
sql = "INSERT INTO MEMBER VALUES('김길동',30,'K@K.COM')"
cursor.execute(sql)
print('수행결과 행수 :',cursor.rowcount)
sql = "INSERT INTO MEMBER VALUES('이길동',25,'l@l.com')"
cursor.execute(sql)
print('수행결과 행수 :',cursor.rowcount)

수행결과 행수 : 1
수행결과 행수 : 1


In [20]:
conn.commit() # (반)conn.rollback()

In [21]:
# select 전송 결과 : cursor 가 가리킴
cursor.execute("SELECT * FROM MEMBER")

In [22]:
# insert, update, delete문 실행 결과: cousor.rowcont
# select문 실행 결과 :
        ## fetchall() : 결과를 모두 받을 때(튜플 list)
        ## fetchone() : 결과를 한행씩 받을 때 (튜플)
        ## fetchmany(n) : 결과를 n행 받을 때 (튜플 list)
print(cursor.fetchall())

[('홍길동', 20, 'h@h.com'), ('김길동', 30, 'K@K.COM'), ('이길동', 25, 'l@l.com')]


In [23]:
print(cursor.fetchall()) # 한번 소요된 cursor 객체는 다시 fetch될 수 없음

[]


In [24]:
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
members = cursor.fetchall()
members

[('홍길동', 20, 'h@h.com'), ('이길동', 25, 'l@l.com'), ('김길동', 30, 'K@K.COM')]

In [25]:
for member in members:
    print(member)

('홍길동', 20, 'h@h.com')
('이길동', 25, 'l@l.com')
('김길동', 30, 'K@K.COM')


In [32]:
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
members = []
while True:
    member = cursor.fetchone()
    if member is None:
        print('데이터 끝')
        break
    members.append(member)

데이터 끝


In [33]:
members

[('홍길동', 20, 'h@h.com'), ('이길동', 25, 'l@l.com'), ('김길동', 30, 'K@K.COM')]

In [35]:
#
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
for member in cursor.fetchmany(2):
    print(member)

('홍길동', 20, 'h@h.com')
('이길동', 25, 'l@l.com')


In [36]:
cursor.description

(('NAME', None, None, None, None, None, None),
 ('AGE', None, None, None, None, None, None),
 ('EMAIL', None, None, None, None, None, None))

In [41]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입'
    def __init__(self, name, age, email):
        self.name = name
        self.age = age
        self.email = email
    def __str__(self):
        return"{}\t{}\t{}".format(self.name, self.age, self.email)
def to_member(*row): # 튜플매개변수를 받아 Member형 객체를 return
    return Member(row[0], row[1], row[2])

In [43]:
member = to_member('홍길동', 20, 'h@h.com')
print(member)

홍길동	20	h@h.com


In [47]:
cursor.execute('SELECT * FROM MEMBER')
member_list = [] # sql문 수행한 결과를 담을 객체 list
members =cursor.fetchall() # 튜플 list
# print(members)
for member in members:
    member_list.append(to_member(*member))

In [48]:
for member in member_list:
    print(member)

홍길동	20	h@h.com
김길동	30	K@K.COM
이길동	25	l@l.com


In [54]:
cursor.close()
conn.close()

ProgrammingError: Cannot operate on a closed database.

## 1.3 SQL 구문에 파라미터 사용하기
- qmark(DB에 따라 불가한 경우 있음)
- named(추천)

In [56]:
conn = sqlite3.connect('data/ch15_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN('홍길동','김길동')")
cursor.fetchall()

[('홍길동', 20, 'h@h.com'), ('김길동', 30, 'K@K.COM')]

In [57]:
name1 = input('검색할 이름1?')
name2 = input('검색할 이름2?')
# cursor.execute("SELECT * FROM MEMBER WHERE NAME IN('"+name1"','"+name2"')")
# cursor.execute("SELECT * FROM MEMBER WHERE NAME IN('{name1}','{name2}')")
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN(?, ?)", (name1, name2))
cursor.fetchall()

검색할 이름1?홍길동
검색항 이름2?김길동


[('홍길동', 20, 'h@h.com'), ('김길동', 30, 'K@K.COM')]